# AMOS 2024 - BARRA2 historical Heatwave Case Study (ANIMATED VERSION)

##### In this notebook we demonstrate the use of BARRA2 data in the NCI Data Collection to explore the NSW heatwave of January 2017.

##### More information on BARRA2 data: https://opus.nci.org.au/pages/viewpage.action?pageId=264241166

##### Before using this notebook, users must join ob53 project via, https://my.nci.org.au/mancini/project/ob53/join

Reanalyses are useful for users to go back in time to look at how a past weather event had occurred and the driving atmospheric processes.

This case study looks at the early 2017 New South Wales heatwaves. 

A special climate statement reporting on the exceptional heat in southeast Australia can be found http://www.bom.gov.au/climate/current/statements/scs61.pdf

In January and February, there were three distinct heatwaves in southeast Australia, with the highest temperatures recorded over 9–12 February 2017. 

There were 3 heatwaves across January and early February 2017 saw unusually high daily maximum and minimum temperatures for at least three consecutive days over large parts of the country. The first heatwave began around 10 January and continued to 14 January. 

The pattern of pressure systems - high pressure system over Tasman Sea and an upper level ridge over central and eastern Australia - drew hot air from central part of the continent.

In [1]:
%matplotlib notebook

In [1]:
import os, sys
user = os.environ['USER']
sys.path.append(f"/scratch/om02/{user}/barpa-barra2-amos2024")

# Import standard python modules
import tempfile
import dask.distributed
import loaddata
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)

# Here we will use the LOADDATA module to simply the loading of the data
import loaddata

In [2]:
# Start a dask client
client = dask.distributed.Client()
client

/opt/conda/envs/datasci/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34081 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/34081/status,
Dashboard: /proxy/34081/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39103,Workers: 7
Dashboard: /proxy/34081/status,Total threads: 7
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:41827,Total threads: 1
Dashboard: /proxy/41895/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:34615,


In [3]:
# In order to highlight the rich data available from BARRA2, this tutorial features animations
# The following settings, as well as "%matplotlib notebook" at the beginning of the note-book, allow these animations to run

from matplotlib.widgets import Slider
from matplotlib.animation import FuncAnimation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.rcParams['animation.embed_limit'] = 2**128
plt.ioff()

### Examine the evolution of the first heatwave in February 2017

The domain and time period are defined based on AMOS2024_exercise_heatwave_BARRA2.ipynb

In [4]:
tstart = "20170208"
tend = "20170214"

loc = (-33.8688, 151.2093)
latmin_c = loc[0] - 1
latmax_c = loc[0] + 1
lonmin_c = loc[1] - 1
lonmax_c = loc[1] + 1

latmin_r = loc[0] - 5
latmax_r = loc[0] + 5
lonmin_r = loc[1] - 5
lonmax_r = loc[1] + 5

Load all the data for different variables

In [5]:
#
# Static variables
#

# Load the orography data
ds_orog = loaddata.load_barra2_data("BARRA-R2", "fx", "orog", 
                                    latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))
# Load land sea mask
ds_lsm = loaddata.load_barra2_data("BARRA-R2", "fx", "sftlf",
                                   latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r))

In [6]:
#
# Time varying variables
#
# If unclear what these variables are, use loaddata.whatis(freq, variable_name)
# 

ds_tasmax = loaddata.load_barra2_data("BARRA-R2", "1hr", "tasmax", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r),
                                      chunks={'time': 'auto'})
ds_uas = loaddata.load_barra2_data("BARRA-R2", "1hr", "uas", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r),
                                      chunks={'time': 'auto'})
ds_vas = loaddata.load_barra2_data("BARRA-R2", "1hr", "vas", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r),
                                      chunks={'time': 'auto'})
ds_cll = loaddata.load_barra2_data("BARRA-R2", "1hr", "cll", 
                                      tstart=tstart, tend=tend,
                                      latrange=(latmin_r, latmax_r),
                                      lonrange=(lonmin_r, lonmax_r),
                                      chunks={'time': 'auto'})

Pre-process the data for plotting

Convert temperature to degC

In [7]:
# Convert temperature from K to degC
ds_tasmax['tasmax' ] = ds_tasmax['tasmax'] - 273.15
ds_tasmax['tasmax'] = ds_tasmax['tasmax'].assign_attrs({"units": "degC"})

(NT, NY, NX) = ds_tasmax['tasmax'].shape

Combine 10 metre wind components, uas and vas, into a single xr.Dataset object so that xarray can plot both as vector field

In [8]:
ds_uv = xr.merge([ds_uas['uas'], ds_vas['vas']])

# Thin the data horizontally for plotting purposes, to avoid having very densed wind vectors
ds_uv_subsampled = ds_uv.isel(lat=range(0, NY, 7), lon=range(0, NX, 7)).compute()

Compute spatial mean of tasmax over the smaller subdomain around Sydney

In [9]:
# Define land sea mask over a small 2x2 degrees bounding box around Sydney
latmin_c = loc[0] - 1
latmax_c = loc[0] + 1
lonmin_c = loc[1] - 1
lonmax_c = loc[1] + 1

ds_lsm_c = ds_lsm.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))
mask_condition = (np.tile(ds_lsm_c['sftlf'], (NT,1,1)) >= 100)

# Truncate over a smaller domain focussing around Sydney
ds_tasmax_c = ds_tasmax.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))

# Apply land sea mask to compute spatial mean over land points only
ds_tasmax_masked = ds_tasmax_c['tasmax'].where(mask_condition)

da_tasmax_spatial_av = ds_tasmax_masked.mean(dim=['lat', 'lon'])

Create animation of the heatwave

Each 3-panel shows,
- Left: Timeseries of tasmax around Sydney
- Middle: Spatial maps of tasmax and 10 m wind
- Right: Spatial maps of cloud cover

In [ ]:
#
# Create an animation we first create the initial frame. 
#

# set up the figure
fig=plt.figure(figsize=(14,5)) # set up figure

#
# Populate the initial frame
#
time_step = 0
t = ds_tasmax['time'][time_step]

#
# First subfigure showing the timeseries of spatial mean tasmax
#
ax1 = plt.subplot(1, 3, 1)
fig1_tasmax = da_tasmax_spatial_av.plot.line()
fig1_dot = ax1.plot(t.data, da_tasmax_spatial_av.data[time_step], 'or')
#  label the time step
ax1.set_title(t.data)

#
# Second subfigure plotting temperature and wind vectors
#
ax2 = plt.subplot(1, 3, 2)
# Plot tasmax as background
fig2_tasmax = ds_tasmax.sel(time=t, method='nearest')['tasmax'].plot(vmin=0, vmax=40, cmap=mpl.cm.RdBu_r, cbar_kwargs={"shrink": 0.5})
# Plot the 10m wind vector
fig2_wind = ds_uv_subsampled.sel(time=t, method='nearest').plot.quiver(x='lon', y='lat', u='uas', v='vas', color='blue')
# Plot surface altitude as contour
fig2_orog = ds_orog['orog'].plot.contour(levels=4, colors='k')
# Indicate where is Sydney
ax2.plot(loc[1], loc[0], 'xr', markersize=15)

#
# Last subfigure plotting low level cloud
#
ax3 = plt.subplot(1, 3, 3)
# Plot cloud 
fig3_cll = ds_cll['cll'].sel(time=t, method='nearest').plot(vmin=0, vmax=100, cmap=mpl.cm.Greys_r, cbar_kwargs={"shrink": 0.5})
# Plot surface altitude as contour
fig3_orog = ds_orog['orog'].plot.contour(levels=4, colors='b')
# Indicate where is Sydney
ax3.plot(loc[1], loc[0], 'xr', markersize=15)
    
fig.tight_layout()

#
# Ceate a function that will update the figure to time_step t
#
def animate(time_step): 
    # Plotting for this time step
    t = ds_tasmax['time'][time_step]
    
    # Update the red dot indicator
    fig1_dot[0].set_xdata(x=t.data)
    fig1_dot[0].set_ydata(y=float(da_tasmax_spatial_av.data[time_step]))
    
    # Update the temperature field
    fig2_tasmax.set_array(ds_tasmax.sel(time=t, method='nearest')['tasmax'].values)
    
    # Update the wind vector field
    fig2_wind.set_UVC(ds_uv_subsampled.sel(time=t, method='nearest')['uas'].values, ds_uv_subsampled.sel(time=t, method='nearest')['vas'].values)
    
    # Update the cloud field
    fig3_cll.set_array(ds_cll['cll'].sel(time=t, method='nearest').values)
    
    ax1.set_title(t.data)
    
#
# Finally, we use FuncAnimation to generate our figure
#
nstep = len(ds_tasmax['time'])
FuncAnimation(fig, animate, frames=nstep, interval=50)  # generate animation

# this may take a while!

### 4. Exercise: Consider a different high impact weather event. 

Examples are,
1. Melbourne dust storm on 8 February 1983, "“everything went black" as the result of dry, cold front crossing Victoria, preceded by hot, gusty northerly winds. https://webarchive.nla.gov.au/awa/20090330051442/http://pandora.nla.gov.au/pan/96122/20090317-1643/www.bom.gov.au/lam/climate/levelthree/c20thc/storm7.html Consider look at temperature (1hr/tas), wind (1hr/uas, vas) and top-layer soil moisture (1hr/mrsos)

2. Sydney hailstorm on 14 April 1999, with most affected areas include south-east suburbs of Kensington, Kingsford, Botany, Mascot, Randwick and Paddington. https://knowledge.aidr.org.au/resources/storm-sydney-1999/ Consider look at precipitation (1hr/pr) and CAPE (1hr/CAPE).

3. Extreme rainfall in NSW over 2021/2022 - compound event where a series of mesoscale rainfall occurred sequentially in the same location. http://www.bom.gov.au/climate/current/statements/scs76.pdf?20220525 Consider look at precipitation (1hr/pr), sea-level pressure (1hr/psl), and soil moisture (1hr/mrsos or 3hr/mrsol)

4. Black summer bushfire 2019/2020 in SE Australia, during September 2019 to February 2020. http://www.bom.gov.au/climate/current/statements/scs73.pdf Consider look at temperature (1hr/tas), relative humidity (1hr/hurs) and wind (1hr/uas, vas).